# Virtualizarr and Coiled - Building a Virtual Zarr store of the Terraclimate dataset


This notebook is an example of using Virtualizarr together with the Python distributed processing framework [Coiled](https://www.coiled.io/) to generate references using [serverless functions](https://docs.coiled.io/user_guide/functions.html). 

**Note:** running this notebook requires a coiled account.


## The dataset
For this example, we are going to create a virtual zarr store from the [Terraclimate](https://www.climatologylab.org/terraclimate.html) dataset. Terraclimate is a monthly dataset spanning 66 years and containing 14 climate and water balance variables. It is made up of 924 individual NetCDF4 files. When represented as an Xarray dataset, it is over 1TB in size.

## Parallelizing `Virtualizarr` reference generation with coiled serverless functions
Coiled serverless functions allow us to easily spin up hundreds of small VMs, which are great for individual file reference generation. We were able to process 924 netCDF files a virtual xarray dataset in about 18 minutes for ~$0.43.

## Requirements

```bash
pip install 'virtualizarr[icechunk,hdf]' coiled jupyter bokeh jupyter-server-proxy obstore
```

## Imports


In [ ]:
import coiled
import icechunk
import numpy as np
import xarray as xr
from obstore.store import from_url

from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry

## Create the Terraclimate variable and year url combinations 
`14 variables * 66 years = 924 NetCDF files`





In [ ]:
tvars = [
    "aet",
    "def",
    "pet",
    "ppt",
    "q",
    "soil",
    "srad",
    "swe",
    "tmax",
    "tmin",
    "vap",
    "ws",
    "vpd",
    "PDSI",
]
min_year = 1958
max_year = 2023
time_list = np.arange(min_year, max_year + 1, 1)

urls = [
    f"https://climate.northwestknowledge.net/TERRACLIMATE-DATA/TerraClimate_{var}_{year}.nc"
    for year in time_list
    for var in tvars
]

### Define our Virtualizarr `Parser` and `ObjectStoreRegistry`

In [ ]:
bucket = "https://climate.northwestknowledge.net"
store = from_url(bucket)
registry = ObjectStoreRegistry({bucket: store})
parser = HDFParser()

## Define the coiled serverless function

### Serverless function setup notes:
- This coiled function is tailored to AWS
- `vm_type=["m8g.medium"]` - This is a small instance, you shouldn't need large machines for reference generation
- `spot_policy="spot_with_fallback"` is cheaper, but might have unintended consequences
- `idle_timeout="10 minutes"` workers will shut down after 10 minutes of inactivity 
- `n_workers=[10, 300]` adaptive scaling between 10 & 100 workers
- `name` [optional] if you want to keep track of your cluster in the coiled dashboard

More details can be found in the [serverless function API](https://docs.coiled.io/user_guide/functions.html#api).

In [ ]:
@coiled.function(
    region="us-west-2",
    vm_type=["m8g.medium"],
    spot_policy="spot_with_fallback",
    arm=True,
    idle_timeout="10 minutes",
    n_workers=[10, 100],
    name="parallel_reference_generation",
)
def process(url):
    vds = open_virtual_dataset(
        url=url,
        parser=parser,
        registry=registry,
        loadable_variables=["time", "lat", "lon", "crs"],
    )
    return vds


# process.map distributes out the input file urls to coiled functions
# retires=10 allows for individual task retires, which can be useful for inconsistent server behavior
results = process.map(urls, retries=10)


## Combine references into virtual dataset

In [ ]:
# extract generator values into a list
vds_list = [result for result in results]

# combine individual refs into a virtual Xarray dataset

mds = xr.combine_by_coords(
    vds_list, combine_attrs="drop_conflicts", coords="minimal", compat="override"
)
mds

## Save the virtual dataset to Icechunk

Now that we have this virtual dataset, we can write it to Icechunk. 

In this example we're creating a local icechunk store, but you could configure it for cloud storage.

In [ ]:
local_storage_conifg = icechunk.local_filesystem_storage("./terraclimate")
repo = icechunk.Repository.open_or_create(local_storage_conifg)
session = repo.writable_session("main")

In [ ]:
config = icechunk.RepositoryConfig.default()
config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(
        url_prefix="https://climate.northwestknowledge.net/",
        store=icechunk.http_store(),
    ),
)


storage = icechunk.in_memory_storage()
repo = icechunk.Repository.create(storage, config)
session = repo.writable_session("main")

mds.vz.to_icechunk(session.store)

snapshot_id = session.commit("terraclimate reference")
print(snapshot_id)

repo.save_config()

## Open the Icechunk store with Xarray

In [ ]:
combined_ds = xr.open_zarr(session.store, consolidated=False, zarr_format=3)

In [ ]:
combined_ds